In [10]:
import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import Informer
from neuralforecast.losses.pytorch import MAE
from sklearn.preprocessing import MinMaxScaler
import pytorch_lightning as pl

In [11]:

# Custom callback to save training losses at each epoch
class SaveTrainingLossCallback(pl.Callback):
    def __init__(self, log_file='epoch_loss_log_window_informer_model_one_year.txt'):
        self.training_losses = []
        self.log_file = log_file
        self.window_number = 0
        with open(self.log_file, 'w') as f:
            f.write('Epoch,Train_Loss,Window\n')

    def on_train_epoch_end(self, trainer, pl_module):
        # Save the training loss at the end of each epoch
        train_loss = trainer.callback_metrics['train_loss'].item()
        self.training_losses.append(train_loss)
        print(f"Epoch {trainer.current_epoch}: Train Loss = {train_loss}")
        
        # Log the loss to the file
        with open(self.log_file, 'a') as f:
            f.write(f'{trainer.current_epoch},{train_loss},{self.window_number}\n')

    def set_window_number(self, window_number):
        self.window_number = window_number

# Initialize callbacks
save_loss_callback = SaveTrainingLossCallback()
pl_trainer_kwargs = {"callbacks": [save_loss_callback], "accelerator": "cpu", "devices": 1}

In [12]:

# Load and preprocess the data
csv_file_path = '/home/raj/Rajarshi/Term Project/rajarshi_code/rajarshi_code/data/SBIN.NS_day_2022.csv'
sbi_data = pd.read_csv(csv_file_path, parse_dates=['Date'])
sbi_data.dropna(inplace=True)
sbi_data.set_index('Date', inplace=True)
sbi_data = sbi_data.asfreq('B', method='pad')

# Create scalers
scaler_close = MinMaxScaler()
sbi_data['Open_Close_Diff'] = sbi_data['Open'] - sbi_data['Close']
sbi_data['Close'] = scaler_close.fit_transform(sbi_data[['Close']])

# Initialize variables
training_end_date = sbi_data.index.max() - pd.DateOffset(years=1)  # Train using last 1 year of data
final_predictions = []

# Define the window management and model training class
class ModelTrainer:
    def __init__(self, data, scaler_close, save_loss_callback, pl_trainer_kwargs):
        self.data = data
        self.scaler_close = scaler_close
        self.save_loss_callback = save_loss_callback
        self.pl_trainer_kwargs = pl_trainer_kwargs

    def train_model(self, train_data, window_number):
        # Set the window number for the callback
        self.save_loss_callback.set_window_number(window_number)

        # Prepare the training data
        Y_train_df = train_data.reset_index().rename(columns={'Date': 'ds', 'Close': 'y'})
        Y_train_df['unique_id'] = 'SBIN'

        # Initialize and train the Informer model (without historical exogenous variables)
        model = Informer(
            h=7,  # Output horizon (prediction length)
            input_size=60,  # Input window size
            hidden_size=128,
            conv_hidden_size=32,
            n_head=4,
            dropout=0.2,
            encoder_layers=2,
            decoder_layers=1,
            factor=3,
            distil=True,
            loss=MAE(),
            learning_rate=5e-4,
            max_steps=250,  # Adjusted as per your code
            **{'callbacks': [self.save_loss_callback]}  # Pass the callback directly here
        )

        # NeuralForecast object to handle model training
        nf = NeuralForecast(models=[model], freq='B')
        nf.fit(df=Y_train_df)

        # Generate future dataframe automatically
        futr_df = nf.make_future_dataframe()

        # Generate predictions
        forecasts = nf.predict(futr_df=futr_df)
        
        pred_values = self.scaler_close.inverse_transform(forecasts[['Informer']].values)
        dates = futr_df['ds']

        return dates, pred_values



In [13]:
# Initialize the trainer
trainer = ModelTrainer(sbi_data, scaler_close, save_loss_callback, pl_trainer_kwargs)

# Training and prediction loop
window_number = 1
while True:
    train_data = sbi_data.loc[:training_end_date]
    print(f"Training window {window_number}: from {train_data.index.min()} to {train_data.index.max()}")

    # Train the model and get predictions
    dates, pred_values = trainer.train_model(train_data, window_number)

    if len(dates) == 0:
        print("No future dates were generated. Exiting the loop.")
        break

    # Store the predictions
    predictions_df = pd.DataFrame({'Date': dates, 'Predicted Value': pred_values.flatten()})
    final_predictions.append(predictions_df)

    # Update training_end_date for the next window only if dates exist
    training_end_date = dates.iloc[-1] if len(dates) > 0 else training_end_date

    # Break if we reach the end of the data
    if training_end_date >= sbi_data.index.max():
        break

    window_number += 1


Seed set to 1
/home/raj/Rajarshi/Term Project/notebook_files/tpvenv/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/raj/Rajarshi/Term Project/notebook_files/tpvenv/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MAE           | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | enc_embedding | DataEmbedding | 384    | train
4 | dec_embedding | DataEmbedding | 384    | train
5 | encoder       | TransEncoder  | 199 K  | train
6 | decoder       | TransDecoder  | 141 K  | train
-----------------------------------

Training window 1: from 1998-11-02 00:00:00 to 2021-12-30 00:00:00
Epoch 249: 100%|██████████| 1/1 [00:03<00:00,  0.27it/s, v_num=115, train_loss_step=0.0298, train_loss_epoch=0.0298]

`Trainer.fit` stopped: `max_steps=250` reached.


Epoch 249: 100%|██████████| 1/1 [00:03<00:00,  0.27it/s, v_num=115, train_loss_step=0.0298, train_loss_epoch=0.0298]

/home/raj/Rajarshi/Term Project/notebook_files/tpvenv/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

`Trainer.fit` stopped: `max_steps=250` reached.


Epoch 249: 100%|██████████| 1/1 [00:02<00:00,  0.43it/s, v_num=262, train_loss_step=0.0329, train_loss_epoch=0.0329]

/home/raj/Rajarshi/Term Project/notebook_files/tpvenv/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 141.63it/s]


/home/raj/Rajarshi/Term Project/notebook_files/tpvenv/lib/python3.10/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [14]:
# Combine predictions and save
all_predictions_df = pd.concat(final_predictions, ignore_index=True)
output_csv_file = 'prediction_using_window_method_informer_model_one_year.csv'
all_predictions_df.to_csv(output_csv_file, index=False)

print(f"Predictions saved to {output_csv_file}")

# Print the logged training losses
print("Training Losses:", save_loss_callback.training_losses)


Predictions saved to prediction_using_window_method_informer_model_one_year.csv
Training Losses: []
